### Differentiable Tessellation

Our approach to differentiable rendering of parametric geometry relies on differentiable tessellation. This notebook introduces the tessellation functions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import trimesh

from drpg import generate_grid_faces, bezier_sphere, SurfaceTessellator, SurfaceType, get_normals_from_tangents

device = torch.device('cpu')

Create a set of parametric patches defined by a control mesh

NOTE: The function name `bezier_sphere` is a bit misleading because it simply creates a set of patches. They *could* represent Bézier surfaces but also NURBS or B-spline surfaces.

In [ ]:
patches = bezier_sphere(n=5, device=device) 

np.random.seed(913)
color = (np.random.rand(patches.F.shape[0], 3) * 255).astype(np.uint8)

C = patches.V[patches.F]

scene = trimesh.Scene()
for i in range(patches.F.shape[0]):
    c = C[i]

    # Create geometry for the control mesh
    f = generate_grid_faces(patches.n, patches.m)
    mesh = trimesh.Trimesh(c.reshape(-1, 3), f, vertex_colors=color[i])

    # Create geometry for the control points
    points = [trimesh.primitives.Sphere(radius=0.01, center=pt).to_mesh() for pt in c.reshape(-1, 3)]
    for point in points:
        point.visual = trimesh.visual.ColorVisuals(vertex_colors=np.tile(color[i:i+1], reps=(point.vertices.shape[0], 1)))

    scene.add_geometry(mesh)
    scene.add_geometry(points)
scene.show()

A `SurfaceTessellator` can be used to tessellate the patches. The tessellator configuration decides if it is a Bézier or NURBS control mesh.

We first tessellate the control mesh assuming Bézier patches. For Bézier patches, the tessellator can compute the underlying tangents and subsequently normals.

In [ ]:
tessellator = SurfaceTessellator(uv_grid_resolution=(32, 32), type=SurfaceType.Bezier)

v, f, tu, tv = tessellator.tessellate(patches, return_per_patch=True)
n            = get_normals_from_tangents(tu, tv)
print(f"{v.shape=}")
print(f"{f.shape=}")
print(f"{tu.shape=}")
print(f"{tv.shape=}")

scene = trimesh.Scene()
for i in range(v.shape[0]):
    scene.add_geometry(trimesh.Trimesh(v[i].reshape(-1, 3), f[0].reshape(-1, 3), vertex_normals=n, vertex_colors=color[i]))
scene.show()

Now tessellate assuming NURBS patches of degree 2. Tangents and normals are not available as they are not supported by the package used for evaluation (NURBS-Diff).

In [ ]:
tessellator = SurfaceTessellator(uv_grid_resolution=(32, 32), type=SurfaceType.NURBS) # <- Tessellate as NURBs surface

v, f, tu, tv = tessellator.tessellate(patches, degree=2, return_per_patch=True)
print(f"{v.shape=}")
print(f"{f.shape=}")
print(f"{tu=}")
print(f"{tv=}")

scene = trimesh.Scene()
for i in range(v.shape[0]):
    scene.add_geometry(trimesh.Trimesh(v[i].reshape(-1, 3), f[0].reshape(-1, 3), vertex_colors=color[i]))
scene.show()

Curves can be similarly tessellated using a `CurveTessellator`.

In [ ]:
from drpg import bezier_helix, ParametricCurves, CurveTessellator, circle_profile

helix = bezier_helix(windings=3, device=device)
curves = ParametricCurves(helix.shape[1], 3, device=device)
curves.add(helix)

tessellator = CurveTessellator((32))
profile     = circle_profile(0.2)

np.random.seed(913)
color = (np.random.rand(curves.F.shape[0], 3) * 255).astype(np.uint8)

C = curves.V[curves.F]

scene = trimesh.Scene()
for i in range(curves.F.shape[0]):
    c = C[i]

    # Create geometry for the control points
    points = [trimesh.primitives.Sphere(radius=0.01, center=pt).to_mesh() for pt in c.reshape(-1, 3)]
    for point in points:
        point.visual = trimesh.visual.ColorVisuals(vertex_colors=np.tile(color[i:i+1], reps=(point.vertices.shape[0], 1)))

    # scene.add_geometry(mesh)
    scene.add_geometry(points)

v, f, _, _, _ = tessellator.tessellate(curves=curves, profile=profile)

scene.add_geometry(trimesh.Trimesh(v.reshape(-1, 3), f.reshape(-1, 3)))

scene.show()